In [41]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [42]:
df_fields = pd.read_csv('../data/fields.csv', delimiter=';', decimal=',', skiprows=[0, 2])
df_value = pd.read_csv('../data/value.csv', delimiter=';', decimal=',', skiprows=[0, 1], parse_dates=['inv_item_date'], date_format={'inv_item_date': '%d.%m.%Y'})
df_works = pd.read_csv('../data/works.csv', delimiter=';', decimal=',', skiprows=[0, 1], parse_dates=['start_date', 'end_date'], date_format={'start_date': '%d.%m.%Y', 'end_date': '%d.%m.%Y'})

df_works['area'] = df_works['area'].str.replace(',', '.').astype(float)

# Формирование таблицы с полями и их yield_plan, yield_fact

In [43]:
valid_work_subtypes = ['Прямое комбайнирование', 'Подбор и обмолот', 'Заготовка сенажа', 'Уборка силос/карнаж', 'Подбор (корм)', 'Вывоз', 'Прессование']
valid_work_types = ['Обработка почвы', 'Уборка']

df_yield_fact_works = df_works.query(
    f"work_type in {valid_work_types} and work_subtype in {valid_work_subtypes}"
)[['season', 'field', 'yield_fact', 'area', 'work_type']]

df_yield_fact = (
    df_yield_fact_works
    .assign(weighted_yield=lambda x: x['yield_fact'] * x['area'])
    .groupby(['field', 'season'])
    .agg(weighted_yield=('weighted_yield', 'sum'), total_area=('area', 'sum'))
    .assign(weighted_mean=lambda x: x['weighted_yield'] / x['total_area'])
    .reset_index()[['field', 'season', 'weighted_mean']]
)

df_yield_fact = df_yield_fact.rename(columns={'weighted_mean': 'yield_fact'})

df_yield_fact.head()

,field,season,yield_fact
0,02.09.02.002.01,2023,1.568170
1,02.09.02.002.01,2024,0.000000
2,02.09.02.015.00,2023,1.223696
3,02.09.02.019.00,2023,2.150000
4,02.09.03.032.00,2023,268.696209


In [44]:
df_fields_tmp = df_fields[['season', 'field', 'p2o5', 'k2o', 'humus', 'ph', 'crop', 'region', 'yield_plan']]

df_yield = df_fields_tmp.merge(df_yield_fact, how='left', on=['field', 'season'])

df_yield = df_yield.dropna()

df_yield.head()

,season,field,p2o5,k2o,humus,ph,crop,region,yield_plan,yield_fact
0,2023,22.47.01.001.01,221.05,206.3,4.05,5.25,Оз.пшеница мягкая,Алтай,3.8,190.592405
3,2023,22.47.01.003.00,186.50,174.8,5.90,5.40,Оз.пшеница мягкая,Алтай,3.8,1.851617
4,2023,22.47.01.004.00,166.60,147.9,5.60,5.40,Оз.пшеница мягкая,Алтай,3.8,1.418186
7,2023,22.47.01.006.02,194.70,195.9,4.00,5.15,Оз.пшеница мягкая,Алтай,3.8,2.187347
9,2023,22.47.01.007.02,184.20,171.4,4.40,5.05,Оз.пшеница мягкая,Алтай,3.8,2.109071


# Добавление информации о товарно-материальных ценностях

In [84]:
df_value.head()

,field,inv_item_type,inv_item_name,inv_item_weight,inv_item_unit,inv_item_dose,inv_item_date,work_id
0,36.01.21.025.01,Удобрения,Удобрение 31,15017.6,кг,260.0,2024-10-28,E3A7DAEB-D124-4685-ADE1-199818C77361
1,36.14.12.064.00,Удобрения,Удобрение 31,6500.0,м3,65.0,2024-10-28,0AB8C8D7-04BD-4483-98B2-3E062CDD7E3E
2,36.14.16.008.00,Удобрения,Удобрение 49,7865.0,т,65.0,2024-10-28,00E51F81-0A4F-4983-B068-877A089FFD33
3,36.14.16.008.00,Удобрения,Удобрение 49,10985.0,т,65.0,2024-10-28,150F0150-8061-4CE9-9570-B112CBAD7C51
4,36.14.16.008.00,Удобрения,Удобрение 49,10985.0,т,65.0,2024-10-28,D5D1EADB-D159-4CF8-8C2A-171517ACD403


In [85]:
df_value['inv_item_type'].value_counts()

inv_item_type
СЗР          100115
Удобрения     43008
Семена        27690
Name: count, dtype: int64

In [86]:
df_fert = df_value[df_value['inv_item_type'] == 'Удобрения']

In [87]:
df_fert.isnull().sum()

field              0
inv_item_type      0
inv_item_name      0
inv_item_weight    0
inv_item_unit      2
inv_item_dose      0
inv_item_date      0
work_id            0
dtype: int64

In [88]:
df_fert['inv_item_unit'].unique()

array(['кг', 'м3', 'т', 'л', nan, 'шт'], dtype=object)

In [89]:
df_fert.shape

(43008, 8)

## В итоге стоит превести всё к общим размерностям:
кг, м3, п.е., п. ед., упак, шт

In [90]:
df_fert = df_fert.dropna()

In [91]:
df_fert['inv_item_unit'].value_counts()

inv_item_unit
кг    31047
т      7862
л      3097
м3      999
шт        1
Name: count, dtype: int64

In [ ]:
df_fert.loc[df_fert['inv_item_unit'] == 'т', 'inv_item_dose'] *= 1000
df_fert.loc[df_fert['inv_item_unit'] == 'т', 'inv_item_unit'] = 'кг'
df_fert.head()

,field,inv_item_type,inv_item_name,inv_item_weight,inv_item_unit,inv_item_dose,inv_item_date,work_id
0,36.01.21.025.01,Удобрения,Удобрение 31,15017.6,кг,260.0,2024-10-28,E3A7DAEB-D124-4685-ADE1-199818C77361
1,36.14.12.064.00,Удобрения,Удобрение 31,6500.0,м3,65.0,2024-10-28,0AB8C8D7-04BD-4483-98B2-3E062CDD7E3E
2,36.14.16.008.00,Удобрения,Удобрение 49,7865.0,кг,65000.0,2024-10-28,00E51F81-0A4F-4983-B068-877A089FFD33
3,36.14.16.008.00,Удобрения,Удобрение 49,10985.0,кг,65000.0,2024-10-28,150F0150-8061-4CE9-9570-B112CBAD7C51
4,36.14.16.008.00,Удобрения,Удобрение 49,10985.0,кг,65000.0,2024-10-28,D5D1EADB-D159-4CF8-8C2A-171517ACD403


In [93]:
df_fert.loc[df_fert['inv_item_unit'] == 'м3', 'inv_item_dose'] *= 1000
df_fert.loc[df_fert['inv_item_unit'] == 'м3', 'inv_item_unit'] = 'л'
df_fert.head()

,field,inv_item_type,inv_item_name,inv_item_weight,inv_item_unit,inv_item_dose,inv_item_date,work_id
0,36.01.21.025.01,Удобрения,Удобрение 31,15017.6,кг,260.0,2024-10-28,E3A7DAEB-D124-4685-ADE1-199818C77361
1,36.14.12.064.00,Удобрения,Удобрение 31,6500.0,л,65000.0,2024-10-28,0AB8C8D7-04BD-4483-98B2-3E062CDD7E3E
2,36.14.16.008.00,Удобрения,Удобрение 49,7865.0,кг,65000.0,2024-10-28,00E51F81-0A4F-4983-B068-877A089FFD33
3,36.14.16.008.00,Удобрения,Удобрение 49,10985.0,кг,65000.0,2024-10-28,150F0150-8061-4CE9-9570-B112CBAD7C51
4,36.14.16.008.00,Удобрения,Удобрение 49,10985.0,кг,65000.0,2024-10-28,D5D1EADB-D159-4CF8-8C2A-171517ACD403


In [98]:
df_fert['inv_item_unit'].value_counts()

inv_item_unit
кг    38909
л      4096
шт        1
Name: count, dtype: int64

In [102]:
r_to_drop = df_fert[df_fert['inv_item_unit'] == 'шт'].index

df_fert = df_fert.drop(r_to_drop)

df_fert['inv_item_unit'].value_counts()

inv_item_unit
кг    38909
л      4096
Name: count, dtype: int64

## Добавление сезона

In [123]:
df_works_tmp = df_works[['field', 'season', 'work_id']].sort_values(by='work_id')
df_works_tmp.head(5)

,field,season,work_id
28762,56.28.06.005.02,2023,0000709B-EDB8-4A6F-A2C2-448AD1A70316
31143,40.23.02.014.01,2023,0000ACB9-08B0-483E-BAFD-C45570C2DA70
107954,36.14.19.030.00,2024,0000BD48-61E4-4F77-8C53-0CC5535655ED
144756,56.28.06.014.00,2024,0000D2BE-1620-4C00-88ED-10CA7C60BCCF
19142,36.14.06.025.00,2023,0000F83C-937B-49B7-AF69-0A573B099D1B


In [121]:
df_fert = df_fert[['field', 'inv_item_name', 'inv_item_weight', 'inv_item_unit', 'work_id']].sort_values(by='work_id')

df_fert.head(5)

,field,inv_item_name,inv_item_weight,inv_item_unit,work_id
113923,36.14.06.025.00,Удобрение 10,44252.33,кг,0000F83C-937B-49B7-AF69-0A573B099D1B
143945,47.22.06.036.02,Удобрение 31,10296.00,л,00022E46-3B96-4FD2-A9F2-10FC62AF4D5E
127023,54.17.12.038.00,Удобрение 49,2293.20,кг,0002E9E7-7E9F-49D3-B189-B4EBD7011E4E
3667,56.28.06.035.02,Удобрение 57,210085.20,кг,00038C65-2CB9-42C3-B51A-95A2A9081B60
44122,16.13.12.055.00,Удобрение 57,914.06,кг,0003C1B2-B2A1-47E8-A43A-3B738A33CB9B


In [124]:
df_fert.merge(df_works_tmp, how='left', on=['field', 'work_id']).isnull().sum()

field                  0
inv_item_name          0
inv_item_weight        0
inv_item_unit          0
work_id                0
season             14722
dtype: int64